<br>
<h1 style = "font-size:40px; font-family:Garamond ; font-weight : normal; background-color: #f6f5f5 ; color : #fe346e; text-align: center; border-radius: 100px 100px;">Telematics : Device Tracking</h1>
<br>

<h1 style="font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; background-color: #ffffff; color: navy;">Introduction&nbsp;&nbsp;&nbsp;&nbsp;</h1> 

For automobile insurers, telematics represents a growing and valuable way to quantify driver risk. Instead of pricing decisions on vehicle and driver characteristics, telematics gives the opportunity to measure the quantity and quality of a driver's behavior. This can lead to savings for safe or infrequent drivers, and transition the burden to policies that represent increased liability.

The intent to understand this is whether a driver drive long trips? Short trips? Highway trips? Back roads? Do they accelerate hard from stops? Do they take turns at high speed? The answers to these questions combine to form an aggregate profile that potentially makes each driver unique.


<h1 style="font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; background-color: #ffffff; color: navy;">Import libraries&nbsp;&nbsp;&nbsp;&nbsp;</h1> 

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly
import plotly.express as ex
import plotly.graph_objs as go
import matplotlib.pyplot as plt
%matplotlib inline
import geopandas as gpd
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import tqdm
from tqdm.notebook import *

<h1 style="font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; background-color: #ffffff; color: navy;">Read Data&nbsp;&nbsp;&nbsp;&nbsp;</h1> 

In [ ]:
data=pd.read_csv('../input/automobile-telematics-dataset/Telematicsdata.csv')

In [ ]:
data.info()

In [ ]:
data.shape

In [ ]:
data.head()

<h1 style="font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; background-color: #ffffff; color: navy;">Separate Position values&nbsp;&nbsp;&nbsp;&nbsp;</h1>

In [ ]:
### Subset for position data
position=data[data['variable']=='POSITION']

In [ ]:
position['deviceId'].unique()

In [ ]:
position.isna().sum()

In [ ]:
position.shape

<h1 style="font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; background-color: #ffffff; color: navy;">Separate Device values&nbsp;&nbsp;&nbsp;&nbsp;</h1>

In [ ]:
## Device id = zRYzhAEAHAABAAAKCRtcAAsAtwB1gBAQ
pos_d1=position[position['deviceId']=='zRYzhAEAHAABAAAKCRtcAAsAtwB1gBAQ']
pos_d2=position[position['deviceId']=='zRYzhAEAHAABAAAKCRtcAAsANAB0gBAQ']
pos_d3=position[position['deviceId']=='zRYzhAEAHAABAAAKCRtcAAsAGAB0gBAQ']

In [ ]:
print(pos_d1.shape)
print(pos_d2.shape)
print(pos_d3.shape)

In [ ]:
pos_d1.sort_values(by='timestamp',ascending=False,inplace=True)
pos_d2.sort_values(by='timestamp',ascending=False,inplace=True)
pos_d3.sort_values(by='timestamp',ascending=False,inplace=True)

In [ ]:
pos_d1['timestamp']=pd.to_datetime(pos_d1['timestamp'])
pos_d2['timestamp']=pd.to_datetime(pos_d2['timestamp'])
pos_d3['timestamp']=pd.to_datetime(pos_d3['timestamp'])

In [ ]:
pos_d1['day']=pos_d1['timestamp'].dt.day
pos_d1['month']=pos_d1['timestamp'].dt.month
pos_d1['year']=pos_d1['timestamp'].dt.year

pos_d2['day']=pos_d2['timestamp'].dt.day
pos_d2['month']=pos_d2['timestamp'].dt.month
pos_d2['year']=pos_d2['timestamp'].dt.year

pos_d3['day']=pos_d3['timestamp'].dt.day
pos_d3['month']=pos_d3['timestamp'].dt.month
pos_d3['year']=pos_d3['timestamp'].dt.year

In [ ]:
pos_d1.reset_index(drop=True,inplace=True)
pos_d2.reset_index(drop=True,inplace=True)
pos_d3.reset_index(drop=True,inplace=True)

<h1 style="font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; background-color: #ffffff; color: navy;">Split Lat , Long and Elevation from Position values&nbsp;&nbsp;&nbsp;&nbsp;</h1>

In [ ]:
temp=pos_d1['value'].str.split(',',expand=True)
temp1=pos_d2['value'].str.split(',',expand=True)
temp2=pos_d3['value'].str.split(',',expand=True)

In [ ]:
temp.rename({0:'start_lat',1:'start_long',2:'start_elevation'},axis=1,inplace=True)
temp.reset_index(drop=True,inplace=True)

temp1.rename({0:'start_lat',1:'start_long',2:'start_elevation'},axis=1,inplace=True)
temp1.reset_index(drop=True,inplace=True)

temp2.rename({0:'start_lat',1:'start_long',2:'start_elevation'},axis=1,inplace=True)
temp2.reset_index(drop=True,inplace=True)

In [ ]:
pos_d1=pd.concat([pos_d1,temp],axis=1)
pos_d2=pd.concat([pos_d2,temp1],axis=1)
pos_d3=pd.concat([pos_d3,temp2],axis=1)

In [ ]:
print(pos_d1.shape)
print(pos_d2.shape)
print(pos_d3.shape)

In [ ]:
pos_d1['start_lat']=pos_d1['start_lat'].astype(float)
pos_d1['start_long']=pos_d1['start_long'].astype(float)
pos_d1['start_elevation']=pos_d1['start_elevation'].astype(float)

pos_d2['start_lat']=pos_d2['start_lat'].astype(float)
pos_d2['start_long']=pos_d2['start_long'].astype(float)
pos_d2['start_elevation']=pos_d2['start_elevation'].astype(float)

pos_d3['start_lat']=pos_d3['start_lat'].astype(float)
pos_d3['start_long']=pos_d3['start_long'].astype(float)
pos_d3['start_elevation']=pos_d3['start_elevation'].astype(float)

In [ ]:
pos_d1['geom'] = pos_d1['start_lat'].map(str) +"," + pos_d1['start_long'].map(str)
pos_d2['geom'] = pos_d2['start_lat'].map(str) +"," + pos_d2['start_long'].map(str)
pos_d3['geom'] = pos_d3['start_lat'].map(str) +"," + pos_d3['start_long'].map(str)

<h1 style="font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; background-color: #ffffff; color: navy;">Scatter Mapbox - Zoom in the map&nbsp;&nbsp;&nbsp;&nbsp;</h1>

In [ ]:
fig=ex.scatter_mapbox(pos_d1, lat=pos_d1['start_lat'], lon=pos_d1['start_long'],hover_data=['timestamp','day','month'],color=pos_d1['day'], zoom=4,opacity=1,size_max=30,title='Location tagging', width=1200,height=800,template="plotly_dark")
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
fig=ex.scatter_mapbox(pos_d2, lat=pos_d2['start_lat'], lon=pos_d2['start_long'],hover_data=['timestamp','day','month'],color=pos_d2['day'], zoom=4,opacity=1,size_max=30,title='Location tagging', width=1200,height=800)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
fig=ex.scatter_mapbox(pos_d3, lat=pos_d3['start_lat'], lon=pos_d3['start_long'],hover_data=['timestamp','day','month'],color=pos_d3['day'], zoom=4,opacity=1,size_max=30,title='Location tagging', width=1000,height=800)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

<h1 style="font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; background-color: #ffffff; color: navy;">Analzing the sensors&nbsp;&nbsp;&nbsp;&nbsp;</h1>

In [ ]:
data['timestamp']=pd.to_datetime(data['timestamp'])

In [ ]:
data['day']=data['timestamp'].dt.day

In [ ]:
t1=data.groupby(['deviceId','variable','day'])['deviceId'].count()
t1=t1.to_frame(name='count')
t1.reset_index(inplace=True)

In [ ]:
## By alaram Class
dv=data.groupby(['deviceId','variable','alarmClass','day','value'])['value'].count()
dv=dv.to_frame(name='count')
dv.reset_index(inplace=True)

<h1 style="font-family: Verdana; font-size: 15px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; background-color: #ffffff; color: red;">Accelerometer-Z&nbsp;&nbsp;&nbsp;&nbsp;</h1>

In [ ]:
ex.scatter(data_frame=dv[(dv['deviceId']=='zRYzhAEAHAABAAAKCRtcAAsAtwB1gBAQ') & (dv['variable']=='ACCELERATION Z')],x='alarmClass',y='value',facet_col='day',hover_name='count',title='Acc Z - Alarm class wise distribution',template="ggplot2")

<h1 style="font-family: Verdana; font-size: 15px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; background-color: #ffffff; color: red;">Accelerometer-Y&nbsp;&nbsp;&nbsp;&nbsp;</h1>

In [ ]:
ex.scatter(data_frame=dv[(dv['deviceId']=='zRYzhAEAHAABAAAKCRtcAAsAtwB1gBAQ') & (dv['variable']=='ACCELERATION Y')],x='alarmClass',y='value',facet_col='day',hover_name='count',title='Acc Y - Alarm class wise distribution',template="ggplot2")

<h1 style="font-family: Verdana; font-size: 15px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; background-color: #ffffff; color: red;">Accelerometer-X&nbsp;&nbsp;&nbsp;&nbsp;</h1>

In [ ]:
ex.scatter(data_frame=dv[(dv['deviceId']=='zRYzhAEAHAABAAAKCRtcAAsAtwB1gBAQ') & (dv['variable']=='ACCELERATION X')],x='alarmClass',y='value',facet_col='day',hover_name='count',title='Acc X - Alarm class wise distribution',template="ggplot2")

<h1 style="font-family: Verdana; font-size: 15px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; background-color: #ffffff; color: red;">Vehicle Speed&nbsp;&nbsp;&nbsp;&nbsp;</h1>

In [ ]:
ex.scatter(data_frame=dv[(dv['deviceId']=='zRYzhAEAHAABAAAKCRtcAAsAtwB1gBAQ') & (dv['variable']=='Vehicle speed')],x='alarmClass',y='value',facet_col='day',hover_name='count',title='Vehicle Speed - Alarm class wise distribution',template="ggplot2")